In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
model_name = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    torch_dtype=torch.float16,
    # load_in_8bit=True,
    # load_in_4bit=True,
    device_map="auto",
    use_cache=True,
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [3]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

from transformers import AutoTokenizer, TextStreamer, pipeline,LlamaForCausalLM,AutoModelForCausalLM
from langchain import HuggingFacePipeline, PromptTemplate
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
print(DEVICE)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=5000,
    do_sample=False,
    repetition_penalty=1.15,
    streamer=streamer
)

llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0.1})

2024-09-12 13:22:03.645601: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


cuda:0


/tmp/ipykernel_296988/2705066920.py:20: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0.1})


In [18]:
person_prompt_tpl="""From the text below written in Portuguese language, extract Entities strictly as instructed below
1. First, look for the Person Entity type in the text and extract the needed information defined below:
   `id` property of each entity must be alphanumeric and must be unique among the entities. You will be referring this property to define the relationship between entities. NEVER create new entity types that aren't mentioned below. Document must be summarized and stored inside Person entity under `description` property
    Entity Types:
    label:'Person',id:string,name:string //Person Node
2. If you cannot find any information on the entities & relationships above, it is okay to return empty value. DO NOT create fictious data
3. Do NOT create duplicate entities
4. Restrict yourself to extract only Person information. No other information should be focussed.
5. NEVER Impute missing values
6. Many names of persosn are preceded by the words "senhor" and "senhora".
Example Output JSON:
{{"entities": [{{"label":"Person","id":"person1","name":"Amelie"}}]}}

Question: Now, extract the Person for the text below -

{text}

Answer:
"""

In [11]:
text="Terça-feira, 15 de setembro de 1863.  Minha querida Amélie, recebi tua carta esta manhã e, embora de minha parte eu não tenha nada de especial para te dizer, não queria demorar para agradecer ao senhor d’Ambel e ao senhor Canu pelo que eles fizeram pela senhora Costeau, pois atuaram como verdadeiros espíritas; a fala deles foi admirável; demonstraram ter bondade, o que não me surpreende, pois a fé que eles têm é sincera. Agradeço, igualmente, ao senhor Vézy.  Em relação à viúva, tu sabes que o senhor Prévent deu-me 200 francos para as boas obras; tu podes valer-te dessa quantia para ajudá-la. Esse dinheiro não poderia ter melhor destino.  Estou entusiasmado com as boas notícias que tu me deste; os Espíritos tinham razão em me dizer que, quando voltasse, eu encontraria um progresso significativo. A comunicação que Erasto me fez através do senhor d’Ambel me é muito encorajadora, e por isso lhe agradeço; a mensagem está de acordo com aquela que A Verdade me comunicou através da senhora Judith.  Tu não me disseste nada a respeito da tua saúde, o que me faz presumir que ela esteja bem, mas /2/ eu ficaria mais tranquilo se tivesse uma confirmação a esse respeito. Quanto a mim, estou muito bem; os últimos banhos que tomei me fortaleceram de uma maneira incrível; não tenho mais aquela fraqueza nas pernas que eu vinha sentindo há muito tempo. Pude provar isso no domingo. O tempo estava muito bom, e eu estava bem tranquilo, ou seja, quase não tive trabalho. Após meu banho, fiz um longo passeio no mar em um pequeno barco a vela e, embora o mar estivesse calmo, pensei que tu não irias gostar de me acompanhar naquela casca de nozes balançando sobre as ondas. Depois do almoço, saí para fazer um passeio até a beira do mar, descendo por entre as rochas que formam o cabo, logo abaixo de onde ficam os sinais e os faróis. Depois de chegar perto do mar, subi até o pequeno vale que antecede os sinais, escalando as rochas e as falésias; e, surpreendentemente, não fiquei nem cansado nem sem fôlego, como eu costumava ficar quando subia uma simples escada. Voltei passando pela capela de Notre Dame des Flots [Nossa Senhora das Ondas], e desci novamente até a beira do mar para jantar no parque das ostras. Voltei para casa às 19h30; estava um pouco cansado e fui deitar; /3/ dormi até as 8h da manhã. Nesses últimos dias, o tempo está excelente, sem nuvens no céu, um sol esplêndido; um mar tranquilo como um lago repleto de navios e embarcações de vários tamanhos: é um espetáculo encantador. <O mar extenso> é para mim uma grande distração; constato a beleza disso a cada dia; não há um ponto distante no horizonte que eu não explore, e frequentemente me dou conta de observações interessantes e instrutivas que me fazem meditar, pois tudo é instrução para quem quer refletir.  O próximo domingo será meu último domingo aqui; terei uma pequena reunião espírita na casa do senhor Rodier, que faz questão de me receber; logo após, farei os preparativos para a minha partida. O dia em que irei embora ainda não está decidido, mas penso que será na terça ou, no mais tardar, na quarta-feira; de qualquer modo, eu te avisarei.  Irei de ônibus, por volta das 10h30 da manhã; assim chegarei às 18h30, uma hora cômoda para todos.  Até breve, querida Amélie,  Carinhosamente,  A.K.  PS: Quando tu me escreveres novamente, tenta me enviar tua carta na manhã de domingo e certifica-te, se possível, de incluir a comunicação de Costeau."

In [9]:
print(text)

Terça-feira, 15 de setembro de 1863.  Minha querida Amélie, recebi tua carta esta manhã e, embora de minha parte eu não tenha nada de especial para te dizer, não queria demorar para agradecer ao senhor d’Ambel e ao senhor Canu pelo que eles fizeram pela senhora Costeau, pois atuaram como verdadeiros espíritas; a fala deles foi admirável; demonstraram ter bondade, o que não me surpreende, pois a fé que eles têm é sincera. Agradeço, igualmente, ao senhor Vézy.  Em relação à viúva, tu sabes que o senhor Prévent deu-me 200 francos para as boas obras; tu podes valer-te dessa quantia para ajudá-la. Esse dinheiro não poderia ter melhor destino.  Estou entusiasmado com as boas notícias que tu me deste; os Espíritos tinham razão em me dizer que, quando voltasse, eu encontraria um progresso significativo. A comunicação que Erasto me fez através do senhor d’Ambel me é muito encorajadora, e por isso lhe agradeço; a mensagem está de acordo com aquela que A Verdade me comunicou através da senhora Ju

In [12]:
from langchain.chains import LLMChain
prompttemplate=PromptTemplate(template=person_prompt_tpl,input_variables=['text'])
chain=LLMChain(llm=llm,prompt=prompttemplate)
import time
t1=time.time()
result=chain(text)
t2=time.time()

print(t2-t1)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{
"entities": [
{
"label": "Person",
"id": "person1",
"name": "Amélie"
},
{
"label": "Person",
"id": "person2",
"name": "senhor d’Ambel"
},
{
"label": "Person",
"id": "person3",
"name": "senhor Canu"
},
{
"label": "Person",
"id": "person4",
"name": "senhora Costeau"
},
{
"label": "Person",
"id": "person5",
"name": "senhor Prevent"
},
{
"label": "Person",
"id": "person6",
"name": "senhor Rodier"
}
]
}

Explanation:
The text mentions several persons named Amélie, senhor d’Ambel, senhor Canu, senhora Costeau, senhor Prevent, and senhor Rodier. We have extracted their names as separate entities with unique ids.
48.36031889915466


In [13]:
place_prompt_tpl="""From the text below written in Portuguese language, extract Entities strictly as instructed below
1. First, look for the Place Entity type in the text and extract the needed information defined below:
   `id` property of each entity must be alphanumeric and must be unique among the entities. You will be referring this property to define the relationship between entities. NEVER create new entity types that aren't mentioned below. Document must be summarized and stored inside Person entity under `description` property
    Entity Types:
    label:'Place',id:string,name:string //Place Node
2. If you cannot find any information on the entities & relationships above, it is okay to return empty value. DO NOT create fictious data
3. Do NOT create duplicate entities
5. Restrict yourself to extract only Person information. No other information should be focussed.
6. NEVER Impute missing values
Example Output JSON:
{{"entities": [{{"label":"Place","id":"place1","name":"Paris"}}]}}

Question: Now, extract the Place for the text below -

{text}

Answer:
"""

In [14]:
from langchain.chains import LLMChain
prompttemplate=PromptTemplate(template=place_prompt_tpl,input_variables=['text'])
chain=LLMChain(llm=llm,prompt=prompttemplate)
import time
t1=time.time()
result=chain(text)
t2=time.time()

print(t2-t1)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{"entities": []}
1.7141742706298828


In [20]:
text=" Contratei uma assistente recomendada pelo senhor Ballet. É uma jovem que me parece bastante adequada. Como ela deve começar apenas na segunda-feira, ainda não a vi trabalhando; mas outra coisa que te fará compreender a necessidade de não prolongar demais a tua ausência, embora eu sinta o quanto a tua presença é agradável e necessária para os teus pais e o quanto a estada no campo pode também ser-te salutar, é que Morin está nos deixando. Seu irmão veio me contar que seus pais lamentam não terem mais seus filhos e que estão os chamando de volta em alto e bom som. Expressei de forma clara o que penso sobre essa maneira de agir e sobre essa instabilidade de ideias. Ela continuará até que tenhamos alguém; mas é preciso substituí-la, o que é muito desagradável, pois ela faz muito bem o seu trabalhinho. Já comecei a procurar, mas ainda não encontrei ninguém; exceto uma jovem de apenas 15 anos, com pouca presença e, aliás, muito instruída; não creio que isso seja conveniente. Além disso, a senhora Vorler está gravemente doente desde ontem, quinta-feira; de modo que hoje estamos com dois professores a menos; assim, as coisas funcionam como podem; felizmente, não há muitas alunas. "

In [21]:
from langchain.chains import LLMChain
prompttemplate=PromptTemplate(template=person_prompt_tpl,input_variables=['text'])
chain=LLMChain(llm=llm,prompt=prompttemplate)
import time
t1=time.time()
result=chain(text)
t2=time.time()

print(t2-t1)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[
 {
   "label": "Person",
   "id": "ballet",
   "name": "Ballet"
 },
 {
   "label": "Person",
   "id": "morin",
   "name": "Morin"
 }
]
13.648610830307007
